# Title
[]()

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *
from chat_functions import *
from langchain.agents import Tool
from data_functions import parse_json_string

In [4]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')

True

In [5]:
models_list = openai_models()

gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
gpt-3.5-turbo-0301
gpt-4-1106-preview
gpt-3.5-turbo-16k-0613
gpt-4-0613
gpt-3.5-turbo-16k
gpt-4
gpt-3.5-turbo
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-0125
gpt-4-vision-preview
gpt-4-0125-preview
gpt-4-turbo-preview


In [2]:
path = '/home/silvhua/repositories/GHL-chat/src/app/private'
contacts = load_json('contacts.json', path)
contactId = contacts['me_fb']

In [3]:
system_message_dict = dict()
conversation_dict = dict()
reply_dict = dict()
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()
inbound_dict = dict()

In [4]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')
response = refresh_token()

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


# Copied from 2024-02-04 notebook

In [ ]:
def create_test_system_message():
    system_message = f"""
You work for an online personal training business. 
Your role is to reach out to leads and book qualified leads for a call to see if they are a good fit 
for the business's online coaching program.
If the contact provides their telephone number, extract that phone number.

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.
- "phone_number" (string or None): The phone number of the contact, if available.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_test_system_message()
    conversation_dict[conversation_id] = create_chatbot(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )

    reply_dict[conversation_id][question_id] = chat_with_chatbot(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]
InboundMessage = 'My number is 7788917189'
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, InboundMessage, tools)

# From 2024-02-10 notebook

In [ ]:
def create_test_system_message(
        business_name='CoachMcloone', 
        prompts_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/prompts',
        examples_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/data/chat_examples', 
        doc_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/data/rag_docs'
        ):
    instructions_filename = f'{business_name}.md'
    examples_filename = f'{business_name}.txt'
    document_filename = f'{business_name}_doc.md'
    try:
        instructions = load_txt(instructions_filename, prompts_filepath)
        examples = load_txt(examples_filename, examples_filepath)
        document = load_txt(document_filename, doc_filepath)
    except Exception as error:
        if cloud == True:
            print(f'Error: {error}')
            print('Loading prompt files from s3...')
        s3 = boto3.client('s3')
        instructions = s3.get_object(
            Bucket='ownitfit-silvhua', Key=instructions_filename
            )['Body'].read().decode('utf-8')
        examples = s3.get_object(
            Bucket='ownitfit-silvhua', Key=examples_filename
            )['Body'].read().decode('utf-8')
        document = s3.get_object(
            Bucket='ownitfit-silvhua', Key=document_filename
            )['Body'].read().decode('utf-8')
    # print(f'**Instructions component of system message**: \n{instructions}\n')

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
If the message indicates the contact has an eating disorder, suicidal ideation, or other serious mental health 
conditions, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.
- "phone_number" (string or None): The phone number of the contact, if available.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Revise your response if needed to avoid asking questions that have already been answered in previous messages.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    ###
    # print(f'\n**System_message**: {system_message}\n\n')
    return system_message

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_test_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )

    reply_dict[conversation_id][question_id] = chat_with_chatbot(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]
inbound_id = 1
inbound_dict[inbound_id] = 'hi'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

# from 2024-02-19 notebook

In [ ]:
def create_chatbot_test(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor( # Example of creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True,
        parser=parser  # Add the parser instance to the agent_executor
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history
    }
    return agent_info

def chat_with_chatbot_test(user_input, agent_info):
    start_time = time()
    chat_history = agent_info['chat_history'].messages
    print(f'Chat history length: {len(chat_history)}')
    print(f'\nChat history:')
    for item in chat_history:
        print(f'**{item.type.upper()}**: {item.content}')
    last_message = chat_history[-1].content
    manychat_outbound_message_substrings = [
        "I'll be in touch as soon as I'm online next!",
        "I'll give you a personal message here shortly."
    ]
    previous_message_type = chat_history[-2].type
    last_message_type = chat_history[-1].type
    past_outbound_messages = [item.content for item in chat_history if item.type.lower() == 'ai']
    print(f'Past outbound messages: {[item for item in past_outbound_messages]}')
    if (last_message == user_input): ## Check that the current user_input is the most recent message        
        # If the last message is also Inbound, then join all inbound messages together and delete them from chat history
        if previous_message_type == 'human': 
            last_inbound_messages_list = []
            for item in chat_history[-1:0:-1]:
                if item.type.lower() == 'human':
                    last_inbound_messages_list.append(item.content)
                    truncated_history = chat_history[:-1]
                else:
                    break
            last_inbound_messages = '\n\n'.join(reversed(last_inbound_messages_list))
            user_input = last_inbound_messages
            print(f'Joining previous messages as full user input: {user_input}')
            chat_history = truncated_history
        else:
            chat_history = chat_history[:-1]

        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')

    elif (last_message_type != 'human') & (any(substring in last_message for substring in manychat_outbound_message_substrings)):
        result = agent_info['agent_executor']({
                "input": user_input,
                "chat_history": chat_history
            })  
        print(f'Agent response time: {time() - start_time} seconds')
    else:
        result = dict()
        result['output'] = '{"response": "Abort Lambda function", "alert_human": false}'
    # Check that the generated response is not similar to a previously sent outbound message.
    for item in past_outbound_messages:
        if item in result['output']:
            result['output'] = '{"response": "[AI response similar to previous outbound message.]", "alert_human": true}'
            break
    return result

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_system_message('CoachMcloone')
    conversation_dict[conversation_id] = create_chatbot_test(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )
    # Add InboundMessage to chat history since this is normally done by Lambda function but not in `chat_functions.py`
    conversation_dict[conversation_id]['chat_history'].add_user_message(InboundMessage)
    reply_dict[conversation_id][question_id] = chat_with_chatbot_test(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"Do not invoke this function.",
    )
]
inbound_id = 1
inbound_dict[inbound_id] = 'can we do a call?'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

Chat history length: 25

Chat history:
**HUMAN**: GET STARTED
**AI**: So tell me, what's the biggest obstacle holding you back from achieving this goal then?
**HUMAN**: 💪 Training
**AI**: Thanks Silvia 🙏 I'll give you a personal message here shortly.

Just quickly, what's your current height and weight right now?

PS: Yes this message is automated, (I know you probably guessed that) but if you reply here, I'll get back to you as soon as I'm online next 😊
**HUMAN**: 55 kg 150cm
**AI**: Thanks for providing your height and weight! So, if everything were to go to plan over the next 3-6 months, what results would you like to achieve exactly? 🤔
**HUMAN**: deadlift 300lbs
**AI**: Ah that's a great goal! Tell me, why is deadlifting 300lbs important to you, and how will that impact your life? 

I'd love to help you out the best that I can!  ❤
**HUMAN**: confidence
**AI**: Thank you for opening up to me! You are not alone there! 
**AI**: A lot of my clients used to struggle with confidence befo

{'response': '[AI response similar to previous outbound message.]',
 'alert_human': True}

# *End of Page*